In [17]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/Datasets/Edited-mpxj 8.0 - mpxj 8.0.csv")

In [21]:
Abhi.head(5)

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,5,29,1,60,5,NO
1,0,0,10,3,3,0,0,NO
2,1,0,8,43,1,70,8,NO
3,2,8,21,792,1,0,5,NO
4,13,0,23,292,1,83,44,YES


In [22]:
Abhi['change'] = Abhi['change'].map({'YES': 1, 'NO': 0})

In [23]:
Abhi.shape

(821, 8)

In [24]:
Abhi.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,5,29,1,60,5,0
1,0,0,10,3,3,0,0,0
2,1,0,8,43,1,70,8,0
3,2,8,21,792,1,0,5,0
4,13,0,23,292,1,83,44,1


In [25]:
X = Abhi.drop('change', axis=1)
y = Abhi['change']

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [27]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = DecisionTreeClassifier()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [28]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [29]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [30]:
models = get_models()

In [31]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.345 (0.476)


In [32]:
print(y_pred)

[0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0
 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 0
 1 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0]


In [33]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[79 36]
 [29 21]]


In [34]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.5841739130434782


In [35]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.10336937438940882


In [36]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5534782608695652


In [37]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.5371421963786077


In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71       115
           1       0.37      0.42      0.39        50

    accuracy                           0.61       165
   macro avg       0.55      0.55      0.55       165
weighted avg       0.62      0.61      0.61       165

